In [26]:
# Initialization
import numpy as np
from numpy import linalg as la
import pandas as pd
from scipy import optimize
import warnings
import math
import sys
from tabulate import tabulate
import csv

import CovidKL
import CovidCI

import imp
imp.reload(CovidCI)
imp.reload(CovidKL)#applies changes made to file

# Read in data
cases = pd.read_csv('CovidStratified/Data/cases.csv', index_col=0)
cases = cases.sort_index(ascending = True)
# to retrieve entries as plain data types: cases.iloc[i,j].values
cases_dict = {}
cases_mat = cases.to_dict("split")["data"] # temp variable to hold onto arrays
for i in range(len(cases)):
    cases_dict[cases.index[i]] = cases_mat[i]
del cases_mat

prem = {
"can" : pd.read_csv('CovidStratified/Data/premCan.csv', index_col=0).to_numpy(),
"chn" : pd.read_csv('CovidStratified/Data/premChn.csv', index_col=0).to_numpy(),
"gbr" : pd.read_csv('CovidStratified/Data/premGbr.csv', index_col=0).to_numpy(),
"isr" : pd.read_csv('CovidStratified/Data/premIsr.csv', index_col=0).to_numpy(),
"ita" : pd.read_csv('CovidStratified/Data/premIta.csv', index_col=0).to_numpy(),
"nld" : pd.read_csv('CovidStratified/Data/premNLD.csv', index_col=0).to_numpy(),
}
# to retrieve entries: prem["can"]

countries = list(prem)

# print(cases)
# print(prem["can"])

In [27]:
# Example of Confidence intervals

import numpy as np               # for numerical operations
from scipy import stats          # for stats functions
from scipy import optimize as op # to maximize the likelihood

import numdifftools as nd        # to compute gradient and Hessian numerically;
                                 # the package can be found on pypi.
                                 # Another good package for that purpose
                                 # (using automatic differentiation) is autograd

from ci_rvm import find_CI
np.random.seed(451)

# Define the size of the data set
n = 100

# Define the true parameters
k, p = 5, 0.1

# Generate the data set
data = np.random.negative_binomial(k, p, size=n)

# Because the parameters are constrained to the positive range and the
# interval (0, 1), respectively, we work on a transformed parameter space
# with unbounded domain.
def transform_parameters(params):
    k, p = params
    return np.exp(k), 1/(1+np.exp(-p))
    #return k, p

# Log-Likelihood function for a negative binomial model
def logL(params):
    k, p = transform_parameters(params)
    return stats.nbinom.logpmf(data, k, p).sum()

# negative log-Likelihood function for optimization (because we use
# minimization algorithms instead of maximization algorithms)
negLogL = lambda params: -logL(params)

# Initial guess
x0 = [4, 0.2]

# Maximize the likelihood
result = op.minimize(negLogL, x0)
#print(result.x)

# Print the result (we need to transform the parameters to the original
# parameter space to make them interpretable)
print("The estimate is: k={:5.3f}, p={:5.3f}".format(*transform_parameters(result.x)))



The estimate is: k=4.233, p=0.086


In [28]:
np.random.seed(451)

# Define gradient and Hessian
jac = nd.Gradient(logL)
hess = nd.Hessian(logL)

# Find confidence intervals for all parameters.
# Note: For complicated problems, it is worthwile doing this in parallel.
#       However, then we would need to encapsulate the procedure in a
#       method and define the likelihood function, gradient, and Hessian
#       on the top level of the module.
CIs = find_CI(result.x, logL, jac, hess, alpha = 0.95,
              disp=False) # the disp argument lets the algorithm print
                         # status messages.
    
#print(result.x)
#print(CIs)
    
# CIs is a 2D numpy array with CIs[i, 0] containing the lower bound of the
# confidence interval for the i-th parameter and CIs[i, 1] containing the
# respective upper bound.

# Print the confidence intervals. Note: we need to transform the parameters
# back to the original parameter space.
original_lower = transform_parameters(CIs[:,0])
original_upper = transform_parameters(CIs[:,1])
print("Confidence interval for k: [{:5.3f}, {:5.3f}]".format(
   original_lower[0], original_upper[0]))
#print("Confidence interval for p: [{:5.3f}, {:5.3f}]".format(
#   original_lower[1], original_upper[1]))
print("\n")
print("trans: k: ", transform_parameters(result.x)[0])
print("trans: k - upper:", transform_parameters(result.x)[0] - original_upper[0])
print("trans: k - lower:", transform_parameters(result.x)[0] - original_lower[0])
print("\n")
print("non-trans: k: ", result.x[0])
print("non-trans: k - upper: ", result.x[0] - CIs[0,0])
print("non-trans: k - lower: ", result.x[0] - CIs[0,1])

Confidence interval for k: [3.106, 5.687]


trans: k:  4.2334692165135435
trans: k - upper: -1.4535306447825596
trans: k - lower: 1.1274530109756


non-trans: k:  1.4430218026703197
non-trans: k - upper:  0.30968086046292687
non-trans: k - lower:  -0.2951610412647001


In [29]:
imp.reload(CovidKL)

starts = 5 #number of different starting points to test
trials = 5 # Number of trials per start point
iters = (starts * trials)
theta_iter = [[0] * 33] * iters
best_res = optimize.OptimizeResult(x = 100, fun = 10000, success = False) #placeholder var to track best result

print ("Cumulative country calculation with randomized starting points:\n")
kwargs = dict(args = (prem, cases_dict, countries), method="L-BFGS-B")
for i in range(0, starts):
    theta_guess = np.random.gamma(4, 1/4, 14) # mean = 1, sd = 2
    #for k in range(len(theta_guess)): theta_guess[k] = math.log(theta_guess[k], math.e)
    for j in range(0, trials):
        res = optimize.basinhopping(CovidKL.Covid_KL, theta_guess, minimizer_kwargs = kwargs)
        out_start = np.insert(np.insert(theta_guess, 2,1),10,1)
        out_end = np.insert(np.insert(res.x, 2,1),10,1)
        theta_iter[((i*trials)+j)] = np.append(np.append(out_start, out_end), res.fun)
        print(res)
        if (res.fun < best_res.fun): 
            best_res = res
            
    print(f"{((((i+1)*trials)/iters)*100):.2f}","%, ", end = "")


print(tabulate(theta_iter))


Cumulative country calculation with randomized starting points:



C:\Users\zakst\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


                        fun: 0.32173121565000024
 lowest_optimization_result:       fun: 0.32173121565000024
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0039697 ,  0.0037738 , -0.00599811, -0.00144573, -0.0044096 ,
       -0.0017532 ,  0.0001822 , -0.00338231, -0.00118667, -0.00698955,
       -0.0015081 , -0.00162346, -0.00079958,  0.00160543])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 165
      nit: 8
     njev: 11
   status: 0
  success: True
        x: array([0.29897039, 0.55974012, 1.20153977, 1.64723415, 1.70971646,
       1.82612793, 1.90862148, 0.87282596, 0.12647942, 0.76317182,
       0.04023664, 1.48629291, 1.83880673, 2.49645172])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 62
                       nfev: 24000
                        nit: 100
                       njev: 1600
                          x: array([0.29897039, 0.5

                        fun: 0.3214413688628149
 lowest_optimization_result:       fun: 0.3214413688628149
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00164218,  0.00092904,  0.00101101, -0.00170674, -0.00457523,
        0.0008833 , -0.00293921, -0.00291206,  0.0004555 , -0.00155445,
        0.00043569, -0.00083055, -0.00164626, -0.00221024])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 210
      nit: 12
     njev: 14
   status: 0
  success: True
        x: array([3.17477730e-01, 4.88809614e-01, 1.28594285e+00, 1.75235652e+00,
       1.82258713e+00, 1.99110159e+00, 2.12249648e+00, 8.54518958e-01,
       4.96389735e-01, 6.34745006e-01, 6.30109289e-04, 1.24719620e+00,
       1.56575099e+00, 1.95008882e+00])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 70
                       nfev: 25395
                        nit: 100
                       

                        fun: 0.32034619490308547
 lowest_optimization_result:       fun: 0.32034619490308547
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.02591098,  0.00666991,  0.00389261, -0.00477088, -0.016463  ,
       -0.00429107, -0.00226039, -0.00085958,  0.00157494,  0.00034695,
        0.00426575, -0.00105956, -0.00021783, -0.00195387])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 195
      nit: 10
     njev: 13
   status: 0
  success: True
        x: array([0.26892408, 0.39972272, 1.14186659, 1.50764695, 1.48954678,
       1.62435157, 1.75348193, 2.06475329, 1.04671533, 0.85559329,
       0.00598488, 2.29458216, 2.56299807, 2.99193621])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 53
                       nfev: 21945
                        nit: 100
                       njev: 1463
                          x: array([0.26892408, 0.

                        fun: 0.3208792251432175
 lowest_optimization_result:       fun: 0.3208792251432175
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00054348,  0.00072456,  0.00273835,  0.00157968, -0.00018919,
       -0.0011401 ,  0.00160575, -0.00282865, -0.00165812, -0.00128176,
        0.00334302,  0.00034801,  0.00133807,  0.00043818])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 195
      nit: 10
     njev: 13
   status: 0
  success: True
        x: array([0.29890655, 0.56171764, 1.2184695 , 1.69844715, 1.76023546,
       1.83344496, 1.97043263, 1.00356184, 0.14341639, 0.82454534,
       0.00921275, 1.4345487 , 1.94379036, 2.38143618])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 65
                       nfev: 25545
                        nit: 100
                       njev: 1703
                          x: array([0.29890655, 0.56

In [30]:
imp.reload(CovidCI)
imp.reload(CovidKL) #applies changes made to file
np.random.seed(451)

best_res_x = best_res.x
best_res_x2 = [0]*14
best_est = [0.219294308, 0.423309799, 1.124597388, 1.511734653, 1.560386081, 1.727919503, 1.734579037, 
            3.77747199,  0.71170154,  0.769347828, 0.104869899, 2.090248998, 1.98923028,  3.373799478]
for i in range(0, len(best_est)):
    best_est[i] = math.log(best_est[i])
    best_res_x2[i] = math.log(best_res.x[i])

# Initialize variables
CovidCI.set_prem(prem)
CovidCI.set_cases(cases_dict)
CovidCI.set_countries(countries)


In [ ]:
cis_2a1 = CovidCI.CI_calc(best_res_x, CovidCI.pos_Covid_KL)
print("\n\n\nDone a1\n\n")
cis_2a2 = CovidCI.CI_calc(best_res_x2, CovidCI.pos_Covid_KL)
#print("\n\n\nDone a2\n\n")
#cis_2a3 = CovidCI.CI_calc(best_res_x, CovidCI.pos_Covid_KL)
#print("\n\n\nDone a3\n\n")

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 805, in find_CI_bound
    precise, fActual, JActual = test_precision()
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 372, in test_precision
    fActual = fun(xTmp)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 246, in <lambda>
    fun = lambda x: fun2(flip(x))
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidCI.py", line 62, in pos_Covid_KL
    return CovidKL.Covid_KL(theta0, prem_global, cases_global, countries_global)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidKL.py", line 135, in Covid_KL
    res = Covid_KL_k(theta_in, prem_in[country_codes[i]], cases_in[country_codes[i]], debug = debug)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Exec

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 805, in find_CI_bound
    precise, fActual, JActual = test_precision()
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 372, in test_precision
    fActual = fun(xTmp)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidCI.py", line 62, in pos_Covid_KL
    return CovidKL.Covid_KL(theta0, prem_global, cases_global, countries_global)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidKL.py", line 135, in Covid_KL
    res = Covid_KL_k(theta_in, prem_in[country_codes[i]], cases_in[country_codes[i]], debug = debug)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidKL.py", line 115, in Covid_KL_k
    u_tilda = CovidEig.scv_eig(s = theta0[0:(int(len(theta0)/2))], c = prem_in, v

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 1214, in find_CI_bound
    H = hess(x)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 248, in <lambda>
    hess = lambda x: flip(np.asarray(hess2(flip(x))))
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 538, in __call__
    return super(Hessdiag, self).__call__(np.atleast_1d(x), *args, **kwds)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 288, in __call__
    results, f_xi = self._derivative(x_i, args, kwds)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 238, in _derivative_nonzero_order
    results = [diff(f, fxi, x_i, h) for h in steps]
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 238, in <li

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidCI.py:46: RuntimeWarning: overflow encountered in exp
  params[i] = np.exp(params_in[i])
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEig.py:72: RuntimeWarning: invalid value encountered in matmul
  scv = np.matmul(np.matmul(s,c),v)


Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 683, in find_CI_bound
    stop, fActual, JActual = test_precision()
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 372, in test_precision
    fActual = fun(xTmp)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 246, in <lambda>
    fun = lambda x: fun2(flip(x))
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidCI.py", line 62, in pos_Covid_KL
    return CovidKL.Covid_KL(theta0, prem_global, cases_global, countries_global)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidKL.py", line 135, in Covid_KL
    res = Covid_KL_k(theta_in, prem_in[country_codes[i]], cases_in[country_codes[i]], debug = debug)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executa

...

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 773, in find_CI_bound
    xTmp_diff, _ = subproblem.solve(tmpRadius,
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 63, in solve
    subproblem = IterativeSubproblem(self.x, self.fun, self.jac, self.hess,
  File "C:\Users\zakst\anaconda3\lib\site-packages\scipy\optimize\_trustregion_exact.py", line 245, in __init__
    self.hess_inf = norm(self.hess, np.Inf)
  File "C:\Users\zakst\anaconda3\lib\site-packages\scipy\linalg\misc.py", line 145, in norm
    a = np.asarray_chkfinite(a)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\function_base.py", line 488, in asarray_chkfinite
    raise ValueError(
ValueError: array must not contain infs or NaNs
Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 805, in find_CI_bound
    precise, fActual, JActual = test_precision()
  File "C:\User

C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\limits.py:150: UserWarning: All-NaN slice encountered
  warnings.warn(str(msg))
C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py:1138: RuntimeWarning: divide by zero encountered in log2
  fImprovement = np.log2(np.abs((f-target)/(fActual-target)))
C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 773, in find_CI_bound
    xTmp_diff, _ = subproblem.solve(tmpRadius,
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 63, in solve
    subproblem = IterativeSubproblem(self.x, self.fun, self.jac, self.hess,
  File "C:\Users\zakst\anaconda3\lib\site-packages\scipy\optimize\_trustregion_exact.py", line 245, in __init__
    self.hess_inf = norm(self.hess, np.Inf)
  File "C:\Users\zakst\anaconda3\lib\site-packages\scipy\linalg\misc.py", line 145, in norm
    a = np.asarray_chkfinite(a)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\function_base.py", line 488, in asarray_chkfinite
    raise ValueError(
ValueError: array must not contain infs or NaNs


C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidCI.py:46: RuntimeWarning: overflow encountered in exp
  params[i] = np.exp(params_in[i])
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEig.py:72: RuntimeWarning: invalid value encountered in matmul
  scv = np.matmul(np.matmul(s,c),v)
C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\limits.py:150: UserWarning: All-NaN slice encountered
  warnings.warn(str(msg))
C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py:1138: RuntimeWarning: divide by zero encountered in log2
  fImprovement = np.log2(np.abs((f-target)/(fActual-target)))
C:\User

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 773, in find_CI_bound
    xTmp_diff, _ = subproblem.solve(tmpRadius,
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 63, in solve
    subproblem = IterativeSubproblem(self.x, self.fun, self.jac, self.hess,
  File "C:\Users\zakst\anaconda3\lib\site-packages\scipy\optimize\_trustregion_exact.py", line 245, in __init__
    self.hess_inf = norm(self.hess, np.Inf)
  File "C:\Users\zakst\anaconda3\lib\site-packages\scipy\linalg\misc.py", line 145, in norm
    a = np.asarray_chkfinite(a)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numpy\lib\function_base.py", line 488, in asarray_chkfinite
    raise ValueError(
ValueError: array must not contain infs or NaNs
Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 805, in find_CI_bound
    precise, fActual, JActual = test_precision()
  File "C:\User

In [ ]:
cis_2b1 = CovidCI.CI_calc(best_est, CovidCI.neg_Covid_KL)
print("\n\n\nDone b1\n\n")
#cis_2b2 = CovidCI.CI_calc(best_est, CovidCI.neg_Covid_KL)
#print("\n\n\nDone b2\n\n")
#cis_2b3 = CovidCI.CI_calc(best_est, CovidCI.neg_Covid_KL)
#print("\n\n\nDone b3\n\n")

In [ ]:
thetas = best_res_x
confint = cis_2a3

print("Kullback-Leibler, Estimate\n")

print("Parameter #1 (non-trans)")
print("non-trans: theta: ", thetas[0])
print("non-trans: CIs:", confint[0,:])
print("non-trans: theta - upper: ", thetas[0] - confint[0,0])
print("non-trans: theta - lower: ", thetas[0] - confint[0,1])
print("\n")

for i in range(0,len(thetas)):
    print("Parameter #", i+1)
    print("trans: theta: ", CovidCI.param_trans(thetas)[i])
    print("trans: CIs:", CovidCI.param_trans(confint[i,:]))
    print("trans: upper - theta:", CovidCI.param_trans(confint[i,1]) - CovidCI.param_trans(thetas)[i])
    print("trans: theta - lower:", CovidCI.param_trans(thetas)[i] - CovidCI.param_trans(confint[i,0]))
    print("\n")

In [ ]:
ci_header = ["Age", "Lower", "Estimate", "Upper"]

ci_res = [["s0-9","s10-19","s30-39","s40-49","s50-59","s60-69","s70-79",
           "v0-9","v10-19","v30-39","v40-49","v50-59","v60-69","v70-79"],
          CovidCI.param_trans2(cis_2b[:,0]),
          CovidCI.param_trans2(best_est),
          CovidCI.param_trans2(cis_2b[:,1])]
print(best_est - cis_2b[:,0])
print(best_est)
print(best_est - cis_2b[:,1])
ci_res = np.array(ci_res).T.tolist()

print(ci_res)
